<a href="https://colab.research.google.com/github/asabbah44/Text_Classification/blob/main/BertCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

In [370]:
!pip install bert-for-tf2
!pip install sentencepiece

In [371]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [372]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

In [485]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/mydataset.csv')


df1 = pd.DataFrame(df1, columns = ['commenttext','label'])
df2 = pd.DataFrame(df2, columns = ['commenttext','label'])

df= df2 #pd.concat([df1,df2])



In [486]:
df.shape

(1513, 2)

In [487]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [488]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [489]:
comments= []
sentences = list(df['commenttext'])
for sen in sentences:
    comments.append(preprocess_text(sen))

In [490]:
df.label.unique()

array(['Documentation', 'Design', 'Test', 'Requirement', 'Defect'],
      dtype=object)

In [491]:
y = df['label']

y = np.array(list(map(lambda x: 0 if x=="Requirement" else 1 if x=="Design" else 2 if x=="Defect" else 3 if x=="Test" else 4, y)))

Creating a BERT Tokenizer

In [492]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [493]:
tokenizer.tokenize("don't need this classificatio")

['don', "'", 't', 'need', 'this', 'class', '##ific', '##ati', '##o']

In [494]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("don't need this class"))

[2123, 1005, 1056, 2342, 2023, 2465]

In [495]:
def tokenize_comments(text_comments):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_comments))

In [464]:
tokenize_comments = [tokenize_comments(comment) for comment in comments]

Prerparing Data For Training


In [496]:
comments_with_len = [[comment, y[i], len(comment)]
                 for i, comment in enumerate(tokenize_comments)]

TypeError: ignored

In [ ]:
print(comments_with_len)

In [ ]:
import random
random.shuffle(comments_with_len)

In [ ]:
print(comments_with_len)

In [469]:
# sort by the len of comments
#comments_with_len.sort(key=lambda x: x[2])

remove the length attribute from all the comments

In [ ]:
sorted_comments_labels = [(comment_lab[0], comment_lab[1]) for comment_lab in comments_with_len]

In [ ]:
# only comment and label sorted 
print(sorted_comments_labels)

convert the sorted dataset into a TensorFlow 2.0-compliant input dataset shape.

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_comments_labels, output_types=(tf.int32, tf.int32))

In [ ]:
processed_dataset

 Pad our dataset for each batch.

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

divide the dataset into test and training sets.

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_comments_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10  # 5= 20% , 10= 10%
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

                                                            

In [ ]:
TOTAL_BATCHES



Creating the model

In [497]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=5,
                 dropout_rate=0.1,
                 training=False,
                
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions,input_length=512)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer4 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=5,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        # l_3 = self.cnn_layer3(l)
        # l_3 = self.pool(l_3) 
        # l_4 = self.cnn_layer4(l)
        # l_4 = self.pool(l_4) 
        
        concatenated = tf.concat([l_1, l_2], axis=-1) # (self.batch_size, 3 * self.cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [498]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 50
CNN_FILTERS = 100
DNN_UNITS = 512
OUTPUT_CLASSES = 5

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [499]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [500]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [501]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
116/116 [==============================] - 5s 38ms/step - loss: 1.0784 - sparse_categorical_accuracy: 0.6454
Epoch 2/5
116/116 [==============================] - 5s 38ms/step - loss: 0.7564 - sparse_categorical_accuracy: 0.7366
Epoch 3/5
116/116 [==============================] - 5s 38ms/step - loss: 0.4995 - sparse_categorical_accuracy: 0.8430
Epoch 4/5
116/116 [==============================] - 5s 38ms/step - loss: 0.2922 - sparse_categorical_accuracy: 0.9103
Epoch 5/5
116/116 [==============================] - 5s 38ms/step - loss: 0.1523 - sparse_categorical_accuracy: 0.9543


In [502]:
results = text_model.evaluate(test_data)
print(results)

12/12 [==============================] - 0s 14ms/step - loss: 0.9842 - sparse_categorical_accuracy: 0.8125
[0.9841931462287903, 0.8125]


In [503]:
results = text_model.evaluate(train_data)
print(results)

116/116 [==============================] - 2s 15ms/step - loss: 0.1076 - sparse_categorical_accuracy: 0.9639
[0.10762815922498703, 0.9639273285865784]
